## Questions

I assume the affiliation should be for the year when the author published/the editor was an editor? Is that available in MAG?

Difference between `OriginalAffiliation` (in PaperAuthorAffiliations) and `LastKnownAffiliationId` (in Authors)? How would I get `OriginalAffiliation` for editors?

### To Do

- Bucketize years for each affiliation bin
- Plot on area chart (separate for now)

In [90]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as so

In [80]:
def rebin(row):
    if row['rank'].isdigit() and 1 <= int(row['rank']) <= 10:
        val = '1-10'
    elif row['rank'].isdigit() and  11 <= int(row['rank']) <= 50:
        val = '11-50'
    elif row['rank'].isdigit() and  51 <= int(row['rank']) <= 100:
        val = '51-100'
    elif (row['rank'] == '101-150')|(row['rank'] == '151-200'):
        val = '101-200'
    elif (row['rank'] == '201-300')|(row['rank'] == '301-400')|(row['rank'] == '401-500'):
        val = '201-500'
    elif (row['rank'] == '501-600')|(row['rank'] == '601-700')|(row['rank'] == '701-800')|(row['rank'] == '801-900')|(row['rank'] == '901-1000'):
        val = '501-1000'
    elif row['rank'] == '1000+':
        val = '1000+'
    else:
        val = 'other'
    return val

In [83]:
def bucketize_affiliations(author_data, editor_data, affiliation_rank):
    authors = pd.read_csv(author_data).drop(labels=['Unnamed: 0'], axis=1)
    editors = pd.read_csv(editor_data).drop(labels=['Unnamed: 0'], axis=1)
    ranks = pd.read_csv(affiliation_rank).drop(labels=['Unnamed: 0'], axis=1)
    
    authors = authors.dropna(subset=['LastKnownAffiliationId'])
    editors = editors.dropna(subset=['LastKnownAffiliationId'])
    
    author_bins = authors.join(ranks.set_index('AffID'), on='LastKnownAffiliationId', how='left', rsuffix='_bin')
    editor_bins = editors.join(ranks.set_index('AffID'), on='LastKnownAffiliationId', how='left', rsuffix='_bin')
    
    author_bins = author_bins.fillna(value={"rank":"1000+"})
    editor_bins = editor_bins.fillna(value={"rank":"1000+"})
    author_bins['newBins'] = author_bins.apply(rebin, axis=1)
    editor_bins['newBins'] = editor_bins.apply(rebin, axis=1)
    
    return author_bins['newBins'].value_counts(), editor_bins['newBins'].value_counts()

In [87]:
def call_bucketize_affiliations(journal):
    return bucketize_affiliations(f"./../author_data/cleaned/{journal}_authors_cleaned.csv",
                                  f"./../mag_processed/random_duplicates/{journal}_random_duplicates.csv",
                                  f"./../normaffil_topuniversities_ranking_2019.csv")
    

In [88]:
aer_authors, aer_editors = call_bucketize_affiliations("aer")

In [92]:
aer_authors

newBins
1-10        91
11-50       86
1000+       84
101-200     38
51-100      38
201-500     38
501-1000    14
Name: count, dtype: int64

In [93]:
plt.stackplot(aer_authors.values, aer_authors.counts)

AttributeError: 'Series' object has no attribute 'counts'